<h4>Lily Grumbach
    <br>M1 Humanités numériques - Université PSL</h4>
<h1><center>Rendu TAIS-TAL </center></h1>
<h2><center>1/2 : NER</center></h2>

    
A partir de ce notebook et des modules créés séparément, l'objectif de ce rendu est de proposer une 

L'étude portera sur deux revues : 
* les <i>Annales d'hygiène et de médecine coloniale</i> (AHMC)
* les <i>Archives de médecine navale </i>(AMN)
Sur une période allant de 1898 à 1908. 

De 1892 à 1898, une seule revue, nommée les <i>Archives de médecine navale et coloniale</i> était l'unique vecteur de communication autant scientifique que corporatif.

Ainsi, étudier les thématiques ainsi que les lieux traités par chacune de ces revues nous permet de nous interroger sur l'évolution et l'indépendantisation de chacune des deux "disciplines". 


Description générale des deux revues de 1898 à 1908 : 



In [36]:
#Généraux : 
import spacy
import fr_core_news_md
import pandas as pd
import numpy as np
import re
#Importation du pretrained model en français
nlp = spacy.load("fr_core_news_md")


from collections import Counter
from os import path
from pathlib import Path
import glob


# 0. Initialisation 

mise en place des formats voulus pour les exploitations à suivre

In [37]:
#Importation de mon document de départ pour le passer en format txt 

###AHMC
dfAHMC = pd.read_csv("./Notebook_data/input/AHMC_articles-originaux_Termine2.tsv" ,sep='\t')

df_spacyAHMC = dfAHMC.loc[dfAHMC["revue_annee"]<= 1908]
df_spacyAHMC = df_spacyAHMC[["article_titre"]]

###AMN
dfAMN=pd.read_csv("./Notebook_data/input/AMN-articles-98-08_clean.tsv",sep="\t")
dfAMN=dfAMN[:-2] #les deux dernières valeurs étaient des nan
df_spacyAMN = dfAMN[["article_titre"]]

##Exportation en format txt : 

##En liste. avec liste[i]=article[i]
txtAHMC_list_path = "./Notebook_data/titres_articles_list_AHMC-to-1908.txt"
txtAMN__list_path = "./Notebook_data/titres_articles_list_AMN-1898-1908.txt"

df_spacyAHMC.to_string(txtAHMC_list_path,index=False)#Exportation des articles en txt
df_spacyAMN.to_string(txtAMN__list_path,index=False)#Exportation des articles en txt

#Avec une seule entrée pour l'ensemble des textes:
txtAHMC_full_path = "./Notebook_data/titres_articles_full_AHMC-to-1908.txt"
txtAMN_full_path = "./Notebook_data/titres_articles_full_AMN-1898-1908.txt"

with open (txtAHMC_full_path,"w") as file_AHMC:
    list_art_AHMC=[]
    for article in df_spacyAHMC.article_titre:
        article=re.sub(r'\d',"",article)
        file_AHMC.write(article)
        list_art_AHMC.append(article)
        
with open (txtAMN_full_path,"w") as file_AMN:
    list_art_AMN=[]
    for article in df_spacyAMN.article_titre:
        article=re.sub(r'\d',"",article)
        file_AMN.write(article)   
        list_art_AMN.append(article)

In [38]:
with open (txtAMN_full_path,"r") as f:
    full_art_AMN =f.readlines()
    if re.search(r'\d{4}',full_art_AMN[0]):
        print("caca")

# 1. Exploration des données

Dans un premier temps, nous allons explorer les vectorisations à priori de chacune des revues pour en explorer les différences d'annotation et de contenu en général. 

## 1.1. CLeaning and organizing data for exploration 

In [39]:
with open (txtAMN_full_path,"r") as f:
    full_art_AMN =f.readlines()

with open (txtAHMC_full_path,"r") as f:
    full_art_AHMC =f.readlines()


In [40]:
#création d'un df pour explorer et comparer engénéral l'ensemble des textes des deux années!

df_comparaison = pd.DataFrame (
{"article_titre":[full_art_AHMC,full_art_AMN]},
    index=["AHMC","AMN"]
)

dfAMN2 = dfAMN.loc[dfAMN["revue_annee"]<= 1908]
dfAMN2 = dfAMN2[["revue_annee","article_titre"]]

#je regroupe les articles par année : 
dfAMN2=dfAMN2.groupby(by="revue_annee")

#Enregistrement du Groupby en DataFrame normal:
dfAMN_par_annee = dfAMN2.sum()
dfAMN_par_annee

#Transformation des en liste de listes
Loflists = dfAMN_par_annee.values.tolist()
dfAMN_par_annee=dfAMN_par_annee.rename(columns={"article_titre":"articles_annee"})


In [41]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common French stop words
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

cv = CountVectorizer(stop_words=list(fr_stop))
data_cv = cv.fit_transform(dfAMN_par_annee.articles_annee)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = dfAMN_par_annee.index
data_dtm

/home/humanum/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,10,14e,1832,1896,1897,1897variétés,1898variétés,19,1900extraction,1900influence,...,étrangerhôpital,étranglée,étrangères,étude,études,étuve,éviter,évolution,île,îles
revue_annee,,,,,,,,,,,,,,,,,,,,,
1898,0,0,0,3,4,1,0,0,0,0,...,0,0,0,0,1,0,2,0,0,0
1899,0,0,0,3,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1900,0,0,0,2,0,0,1,0,0,0,...,0,0,0,0,0,1,0,2,0,0
1901,0,0,0,0,2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1902,0,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
1903,0,1,2,0,0,0,0,0,0,0,...,0,1,1,2,0,0,0,0,0,0
1904,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,1,1,4,0
1905,0,0,0,0,0,0,0,1,0,0,...,1,0,0,4,0,0,0,0,0,0
1906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [42]:
import pickle
#on extrait la matrice de mots :
data_dtm.to_pickle("./Notebook_data/dtm.pkl")

## 1.2. Exploratory data analysis 

On cherche ici à avoir une vision globale de nos données pour mieux les appréhender. On va s'intéresser à deux données : 
1. les mots les plus courants (que nous visualiserons enbar plot)
2. la taille du vocabulaire 


In [43]:
#Transposition pour une meilleure exploitation
data = data_dtm.transpose() 

In [44]:
# Les 30 mots les plus courants pour chaque annéee : 
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(30)
    top_dict[c]= list(zip(top.index, top.values))

top_dict


{1898: [('médical', 6),
  ('hygiène', 6),
  ('islande', 5),
  ('secours', 5),
  ('blessés', 5),
  ('cas', 5),
  ('pêcheurs', 5),
  ('marine', 5),
  ('chastang', 4),
  ('1897', 4),
  ('médicale', 4),
  ('auffret', 4),
  ('bord', 4),
  ('campagne', 4),
  ('transport', 3),
  ('avril', 3),
  ('métal', 3),
  ('madagascar', 3),
  ('1896', 3),
  ('opératoires', 3),
  ('pathologie', 3),
  ('statistique', 3),
  ('moyens', 3),
  ('traitement', 3),
  ('salles', 3),
  ('assistance', 2),
  ('globig', 2),
  ('tuberculose', 2),
  ('allemande', 2),
  ('hommes', 2)],
 1899: [('hygiène', 6),
  ('marine', 5),
  ('médicale', 5),
  ('étiologie', 5),
  ('béribéri', 4),
  ('statistique', 4),
  ('hôpital', 4),
  ('santé', 4),
  ('indes', 3),
  ('médecine', 3),
  ('épidémie', 3),
  ('préventive', 3),
  ('1896', 3),
  ('saint', 3),
  ('rochefort', 3),
  ('bois', 3),
  ('néerlandaises', 3),
  ('procédé', 3),
  ('traitement', 3),
  ('blessés', 3),
  ('service', 3),
  ('poste', 2),
  ('pathologie', 2),
  ('cure', 

In [45]:
# Les 15 mots les plus courants pour chaque annéee : 
for annee, top_words in top_dict.items():
    print(annee)
    print(', '.join([word for word, count in top_words[0:14]]))
    print('---')

1898
médical, hygiène, islande, secours, blessés, cas, pêcheurs, marine, chastang, 1897, médicale, auffret, bord, campagne
---
1899
hygiène, marine, médicale, étiologie, béribéri, statistique, hôpital, santé, indes, médecine, épidémie, préventive, 1896, saint
---
1900
marine, kitasato, paludisme, traitement, médical, bacille, fièvre, yersin, cas, émily, stationné, inspection, navale, médicaments
---
1901
marine, gros, service, statistique, fièvre, maurel, bord, blessés, santé, gazeau, médicale, campagne, recueillies, combat
---
1902
médical, hygiène, médecine, corps, campagne, chine, service, sac, santé, cas, paludisme, ambulance, barbe, duclot
---
1903
marine, bord, hôpitaux, pelletan, contre, camille, eau, officiers, service, lutte, navires, délivrance, hygiène, décret
---
1904
pelletan, eau, marine, hôpital, hôpitaux, tuberculose, médecine, bord, maritime, navale, médicale, école, potable, dépêche
---
1905
thomson, gaston, bord, hygiène, maritime, décret, cas, service, croiseur, ble

<b>Commentaire</b> : 
On remarques des termes qui ne vont pas être pertinents pour l'analyse : dr, variétés, médical, médecine, note(s),rapport,extrait(s), année, service, mois, étude, contribution. 
Ainsi que les dates (et les dates car on s'intéresse vraiment aux sujets).

On va donc les ajouter à la liste des stopwords

In [46]:
#Ajouter à la liste stopwords les mots vides de sens pour notre étude
from collections import Counter

# on récupère dans une même liste les 30 mots les plus courants pour chaque année
words = []
for annee in data.columns:
    top = [word for (word, count) in top_dict[annee]]
    for t in top:
        words.append(t)
        
words

['médical',
 'hygiène',
 'islande',
 'secours',
 'blessés',
 'cas',
 'pêcheurs',
 'marine',
 'chastang',
 '1897',
 'médicale',
 'auffret',
 'bord',
 'campagne',
 'transport',
 'avril',
 'métal',
 'madagascar',
 '1896',
 'opératoires',
 'pathologie',
 'statistique',
 'moyens',
 'traitement',
 'salles',
 'assistance',
 'globig',
 'tuberculose',
 'allemande',
 'hommes',
 'hygiène',
 'marine',
 'médicale',
 'étiologie',
 'béribéri',
 'statistique',
 'hôpital',
 'santé',
 'indes',
 'médecine',
 'épidémie',
 'préventive',
 '1896',
 'saint',
 'rochefort',
 'bois',
 'néerlandaises',
 'procédé',
 'traitement',
 'blessés',
 'service',
 'poste',
 'pathologie',
 'cure',
 'kaiserlichen',
 'bord',
 'inde',
 'für',
 'sevrin',
 'ration',
 'marine',
 'kitasato',
 'paludisme',
 'traitement',
 'médical',
 'bacille',
 'fièvre',
 'yersin',
 'cas',
 'émily',
 'stationné',
 'inspection',
 'navale',
 'médicaments',
 'reinhold',
 'générale',
 'japonaise',
 'kouang',
 'marchand',
 'fachoda',
 'ruge',
 'coup',
 

In [47]:
#On agrège la liste pour voir les plus courants : 
Counter(words).most_common()

[('marine', 10),
 ('bord', 9),
 ('hygiène', 8),
 ('cas', 8),
 ('service', 7),
 ('médicale', 6),
 ('traitement', 6),
 ('hôpital', 6),
 ('tuberculose', 5),
 ('médecine', 5),
 ('maritime', 5),
 ('blessés', 4),
 ('campagne', 4),
 ('statistique', 4),
 ('santé', 4),
 ('fièvre', 4),
 ('navale', 4),
 ('médical', 3),
 ('secours', 3),
 ('guérison', 3),
 ('hôpitaux', 3),
 ('anglais', 3),
 ('eau', 3),
 ('école', 3),
 ('croiseur', 3),
 ('chastang', 2),
 ('avril', 2),
 ('1896', 2),
 ('pathologie', 2),
 ('assistance', 2),
 ('étiologie', 2),
 ('épidémie', 2),
 ('procédé', 2),
 ('paludisme', 2),
 ('médicaments', 2),
 ('brest', 2),
 ('mer', 2),
 ('corps', 2),
 ('barbe', 2),
 ('pelletan', 2),
 ('camille', 2),
 ('navires', 2),
 ('décret', 2),
 ('guerre', 2),
 ('thomson', 2),
 ('gaston', 2),
 ('cuirassé', 2),
 ('étude', 2),
 ('cazamian', 2),
 ('foie', 2),
 ('abcès', 2),
 ('islande', 1),
 ('pêcheurs', 1),
 ('1897', 1),
 ('auffret', 1),
 ('transport', 1),
 ('métal', 1),
 ('madagascar', 1),
 ('opératoires', 1

In [48]:
#Notre observation précédente est confirmée, on crée une nouvelle liste de stop words:
from spacy.lang.fr.stop_words import STOP_WORDS

add_stop_words = ["dr","variétés","note","notes","rapport","année","mois","contribution",
                      "1898","1899","1900","1901","1902","1903","1904","1905","1906","1907","1908","100","11e","1393","143","14e""15","150","15th","16","1832"
                     "14e","15","1836","1839","1843","1884","1893","1894",'1895',"1895notes"]
for word in add_stop_words:
    STOP_WORDS.add(word)
print(type(STOP_WORDS))

<class 'set'>


In [49]:
# On met à jour la matrice de fréquence de mots avec les nouveaux StopWords
from sklearn.feature_extraction.text import CountVectorizer

# On revient au df par année de départ : 
stop = list(fr_stop)
dfAMN_par_annee["articles_annee"]=dfAMN_par_annee["articles_annee"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Recreate document-term matrix
cv = CountVectorizer(stop_words=STOP_WORDS)
data_cv = cv.fit_transform(dfAMN_par_annee.articles_annee)
data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_stop.index = dfAMN_par_annee.index

# Pickle it for later use
import pickle
pickle.dump(cv, open("cv_stop.pkl", "wb"))
data_stop.to_pickle("dtm_stop.pkl")

/home/humanum/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [50]:
data_stop

,10,14e,1832,1896,1897,1897variétés,1898variétés,19,1900extraction,1900influence,...,étrangerhôpital,étranglée,étrangères,étude,études,étuve,éviter,évolution,île,îles
revue_annee,,,,,,,,,,,,,,,,,,,,,
1898,0,0,0,3,4,1,0,0,0,0,...,0,0,0,0,1,0,2,0,0,0
1899,0,0,0,3,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1900,0,0,0,2,0,0,1,0,0,0,...,0,0,0,0,0,1,0,2,0,0
1901,0,0,0,0,2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1902,0,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
1903,0,1,2,0,0,0,0,0,0,0,...,0,1,1,2,0,0,0,0,0,0
1904,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,1,1,4,0
1905,0,0,0,0,0,0,0,1,0,0,...,1,0,0,4,0,0,0,0,0,0
1906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [51]:
#On jette un oeil à nouveau aux 30 mots les plus courants pour chaque annéee : 
data_check=data_stop.transpose()

top_dict_addstop = {}
for c in data_check.columns:
    top = data_check[c].sort_values(ascending=False).head(30)
    top_dict_addstop[c]= list(zip(top.index, top.values))
top_dict_addstop

{1898: [('médical', 6),
  ('hygiène', 6),
  ('islande', 5),
  ('secours', 5),
  ('blessés', 5),
  ('cas', 5),
  ('pêcheurs', 5),
  ('marine', 5),
  ('chastang', 4),
  ('1897', 4),
  ('médicale', 4),
  ('auffret', 4),
  ('bord', 4),
  ('campagne', 4),
  ('transport', 3),
  ('avril', 3),
  ('métal', 3),
  ('madagascar', 3),
  ('1896', 3),
  ('opératoires', 3),
  ('pathologie', 3),
  ('statistique', 3),
  ('moyens', 3),
  ('traitement', 3),
  ('salles', 3),
  ('assistance', 2),
  ('globig', 2),
  ('tuberculose', 2),
  ('allemande', 2),
  ('hommes', 2)],
 1899: [('hygiène', 6),
  ('marine', 5),
  ('médicale', 5),
  ('étiologie', 5),
  ('béribéri', 4),
  ('statistique', 4),
  ('hôpital', 4),
  ('santé', 4),
  ('indes', 3),
  ('médecine', 3),
  ('épidémie', 3),
  ('préventive', 3),
  ('1896', 3),
  ('saint', 3),
  ('rochefort', 3),
  ('bois', 3),
  ('néerlandaises', 3),
  ('procédé', 3),
  ('traitement', 3),
  ('blessés', 3),
  ('service', 3),
  ('poste', 2),
  ('pathologie', 2),
  ('cure', 

In [52]:
#extraire la liste des 30 mots les plus fréquents pour chaque année
words_addstop = []
for annee in data_check.columns:
    top = [word for (word, count) in top_dict_addstop[annee]]
    for t in top:
        words_addstop.append(t)
        
words_addstop


['médical',
 'hygiène',
 'islande',
 'secours',
 'blessés',
 'cas',
 'pêcheurs',
 'marine',
 'chastang',
 '1897',
 'médicale',
 'auffret',
 'bord',
 'campagne',
 'transport',
 'avril',
 'métal',
 'madagascar',
 '1896',
 'opératoires',
 'pathologie',
 'statistique',
 'moyens',
 'traitement',
 'salles',
 'assistance',
 'globig',
 'tuberculose',
 'allemande',
 'hommes',
 'hygiène',
 'marine',
 'médicale',
 'étiologie',
 'béribéri',
 'statistique',
 'hôpital',
 'santé',
 'indes',
 'médecine',
 'épidémie',
 'préventive',
 '1896',
 'saint',
 'rochefort',
 'bois',
 'néerlandaises',
 'procédé',
 'traitement',
 'blessés',
 'service',
 'poste',
 'pathologie',
 'cure',
 'kaiserlichen',
 'bord',
 'inde',
 'für',
 'sevrin',
 'ration',
 'marine',
 'kitasato',
 'paludisme',
 'traitement',
 'médical',
 'bacille',
 'fièvre',
 'yersin',
 'cas',
 'émily',
 'stationné',
 'inspection',
 'navale',
 'médicaments',
 'reinhold',
 'générale',
 'japonaise',
 'kouang',
 'marchand',
 'fachoda',
 'ruge',
 'coup',
 

In [53]:
# Les 15 mots les plus courants pour chaque annéee : 
for annee, top_words in top_dict_addstop.items():
    print(annee)
    print(', '.join([word for word, count in top_words[0:19]]))
    print('---')

1898
médical, hygiène, islande, secours, blessés, cas, pêcheurs, marine, chastang, 1897, médicale, auffret, bord, campagne, transport, avril, métal, madagascar, 1896
---
1899
hygiène, marine, médicale, étiologie, béribéri, statistique, hôpital, santé, indes, médecine, épidémie, préventive, 1896, saint, rochefort, bois, néerlandaises, procédé, traitement
---
1900
marine, kitasato, paludisme, traitement, médical, bacille, fièvre, yersin, cas, émily, stationné, inspection, navale, médicaments, reinhold, générale, japonaise, kouang, marchand
---
1901
marine, gros, service, statistique, fièvre, maurel, bord, blessés, santé, gazeau, médicale, campagne, recueillies, combat, hygiène, secours, équipages, nord, saisons
---
1902
médical, hygiène, médecine, corps, campagne, chine, service, sac, santé, cas, paludisme, ambulance, barbe, duclot, hôpitaux, médicales, médicale, abbatucci, toxi
---
1903
marine, bord, hôpitaux, pelletan, contre, camille, eau, officiers, service, lutte, navires, délivranc

### 1.2. visualisation des résultats

In [54]:
from matplotlib import pyplot as plt

In [55]:
#On crée un nouveau df avec les 20 principaux mots de chaque année
for annee, top_words in top_dict_addstop.items():
    print(annee)
    print(', '.join([word for word, count in top_words[0:19]]))
    print('---')

1898
médical, hygiène, islande, secours, blessés, cas, pêcheurs, marine, chastang, 1897, médicale, auffret, bord, campagne, transport, avril, métal, madagascar, 1896
---
1899
hygiène, marine, médicale, étiologie, béribéri, statistique, hôpital, santé, indes, médecine, épidémie, préventive, 1896, saint, rochefort, bois, néerlandaises, procédé, traitement
---
1900
marine, kitasato, paludisme, traitement, médical, bacille, fièvre, yersin, cas, émily, stationné, inspection, navale, médicaments, reinhold, générale, japonaise, kouang, marchand
---
1901
marine, gros, service, statistique, fièvre, maurel, bord, blessés, santé, gazeau, médicale, campagne, recueillies, combat, hygiène, secours, équipages, nord, saisons
---
1902
médical, hygiène, médecine, corps, campagne, chine, service, sac, santé, cas, paludisme, ambulance, barbe, duclot, hôpitaux, médicales, médicale, abbatucci, toxi
---
1903
marine, bord, hôpitaux, pelletan, contre, camille, eau, officiers, service, lutte, navires, délivranc

In [60]:
df_FreqMots = pd.DataFrame.from_dict(top_dict_addstop)
df_FreqMots2=df_FreqMots.transpose()
df_FreqMots

,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908
0,"(médical, 6)","(hygiène, 6)","(marine, 7)","(marine, 10)","(médical, 4)","(marine, 14)","(pelletan, 13)","(thomson, 12)","(tuberculose, 5)","(bord, 12)","(docteur, 11)"
1,"(hygiène, 6)","(marine, 5)","(kitasato, 4)","(gros, 7)","(hygiène, 4)","(bord, 10)","(eau, 9)","(gaston, 12)","(thomson, 4)","(marine, 8)","(bord, 7)"
2,"(islande, 5)","(médicale, 5)","(paludisme, 4)","(service, 6)","(médecine, 4)","(hôpitaux, 10)","(marine, 8)","(bord, 9)","(pulmonaire, 4)","(foie, 6)","(cas, 6)"
3,"(secours, 5)","(étiologie, 5)","(traitement, 4)","(statistique, 5)","(corps, 3)","(pelletan, 9)","(hôpital, 7)","(hygiène, 7)","(hôpital, 4)","(traduit, 4)","(abcès, 6)"
4,"(blessés, 5)","(béribéri, 4)","(médical, 4)","(fièvre, 5)","(campagne, 3)","(contre, 8)","(hôpitaux, 7)","(maritime, 6)","(cas, 4)","(nouveau, 4)","(étude, 6)"
5,"(cas, 5)","(statistique, 4)","(bacille, 4)","(maurel, 4)","(chine, 3)","(camille, 8)","(tuberculose, 6)","(décret, 5)","(machenaud, 4)","(abcès, 4)","(procédé, 5)"
6,"(pêcheurs, 5)","(hôpital, 4)","(fièvre, 4)","(bord, 4)","(service, 3)","(eau, 7)","(médecine, 6)","(cas, 5)","(temps, 4)","(navale, 4)","(croiseurs, 4)"
7,"(marine, 5)","(santé, 4)","(yersin, 3)","(blessés, 4)","(sac, 3)","(officiers, 7)","(bord, 6)","(service, 5)","(brunet, 4)","(conférence, 3)","(marine, 4)"
8,"(chastang, 4)","(indes, 3)","(cas, 3)","(santé, 4)","(santé, 3)","(service, 6)","(maritime, 6)","(croiseur, 5)","(juillet, 4)","(aspirants, 3)","(foie, 4)"
9,"(1897, 4)","(médecine, 3)","(émily, 2)","(gazeau, 3)","(cas, 3)","(lutte, 6)","(navale, 5)","(blessés, 4)","(marins, 4)","(gauche, 3)","(hôpital, 4)"


In [69]:
def empty_multiindex(names):
    """
    Creates empty MultiIndex from a list of level names.
    """
    return MultiIndex.from_tuples(tuples=[(None,) * len(names)], names=names)

names= ["mot","occurence"]

df2=pd.MultiIndex.from_tuples(tuples=[(None,) * len(names)], names=["mot","occurence"])

for annee, top_words in top_dict_addstop.items():
        df=pd.MultiIndex.from_tuples(top_words,names=["mot","occurence"])
        df2=pd.concat([df2["mot","occurence"],df["mot","occurence"]])
        print(df)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [70]:
top_dict_addstop

{1898: [('médical', 6),
  ('hygiène', 6),
  ('islande', 5),
  ('secours', 5),
  ('blessés', 5),
  ('cas', 5),
  ('pêcheurs', 5),
  ('marine', 5),
  ('chastang', 4),
  ('1897', 4),
  ('médicale', 4),
  ('auffret', 4),
  ('bord', 4),
  ('campagne', 4),
  ('transport', 3),
  ('avril', 3),
  ('métal', 3),
  ('madagascar', 3),
  ('1896', 3),
  ('opératoires', 3),
  ('pathologie', 3),
  ('statistique', 3),
  ('moyens', 3),
  ('traitement', 3),
  ('salles', 3),
  ('assistance', 2),
  ('globig', 2),
  ('tuberculose', 2),
  ('allemande', 2),
  ('hommes', 2)],
 1899: [('hygiène', 6),
  ('marine', 5),
  ('médicale', 5),
  ('étiologie', 5),
  ('béribéri', 4),
  ('statistique', 4),
  ('hôpital', 4),
  ('santé', 4),
  ('indes', 3),
  ('médecine', 3),
  ('épidémie', 3),
  ('préventive', 3),
  ('1896', 3),
  ('saint', 3),
  ('rochefort', 3),
  ('bois', 3),
  ('néerlandaises', 3),
  ('procédé', 3),
  ('traitement', 3),
  ('blessés', 3),
  ('service', 3),
  ('poste', 2),
  ('pathologie', 2),
  ('cure', 

In [136]:
for annee, top_words in top_dict_addstop.items():
    print(annee)
    print(', '.join([word for word, count in top_words[0:20]]))
    print('---')

1898
médical, hygiène, islande, secours, blessés, cas, pêcheurs, marine, chastang, 1897, médicale, auffret, bord, campagne, transport, avril, métal, madagascar, 1896, opératoires
---
1899
hygiène, marine, médicale, étiologie, béribéri, statistique, hôpital, santé, indes, médecine, épidémie, préventive, 1896, saint, rochefort, bois, néerlandaises, procédé, traitement, blessés
---
1900
marine, kitasato, paludisme, traitement, médical, bacille, fièvre, yersin, cas, émily, stationné, inspection, navale, médicaments, reinhold, générale, japonaise, kouang, marchand, fachoda
---
1901
marine, gros, service, statistique, fièvre, maurel, bord, blessés, santé, gazeau, médicale, campagne, recueillies, combat, hygiène, secours, équipages, nord, saisons, anglaise
---
1902
médical, hygiène, médecine, corps, campagne, chine, service, sac, santé, cas, paludisme, ambulance, barbe, duclot, hôpitaux, médicales, médicale, abbatucci, toxi, hôpital
---
1903
marine, bord, hôpitaux, pelletan, contre, camille, 

In [154]:
L_mots = []
L_count = []
L_annee = []
dico_content={}
dico_mots = {}
dico_poidsmots ={}
mytuple = ()
ranks=list(range(20))
df_final=pd.DataFrame({"rank":ranks})
for annee, top_words in top_dict_addstop.items():
    dico_content[annee]=[]
    dico_mots[annee]=[]
    for word, count in top_words[0:20]: 
        mytuple = (annee,words)
        dico_poidsmots[word]=[count]
        dico_content[annee].append(count)
        dico_mots[annee].append(word)
        L_count.append(count)
        L_mots.append(word)
    df=pd.DataFrame(
            {"rank":ranks,annee:L_count},
        index=L_mots
    )
    print(df)
    df_final=pd.merge(df,df_final,how="inner",on="rank")
    print(df_final)

             rank  1898
médical         0     6
hygiène         1     6
islande         2     5
secours         3     5
blessés         4     5
cas             5     5
pêcheurs        6     5
marine          7     5
chastang        8     4
1897            9     4
médicale       10     4
auffret        11     4
bord           12     4
campagne       13     4
transport      14     3
avril          15     3
métal          16     3
madagascar     17     3
1896           18     3
opératoires    19     3
    rank  1898
0      0     6
1      1     6
2      2     5
3      3     5
4      4     5
5      5     5
6      6     5
7      7     5
8      8     4
9      9     4
10    10     4
11    11     4
12    12     4
13    13     4
14    14     3
15    15     3
16    16     3
17    17     3
18    18     3
19    19     3


ValueError: Shape of passed values is (20, 2), indices imply (40, 2)

In [142]:
ranks=list(range(20))
df_final=pd.DataFrame({"rank":ranks})

In [145]:
df_final.shape

(20, 1)

In [124]:
df

,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908
0,"(médical, 6)","(hygiène, 6)","(marine, 7)","(marine, 10)","(médical, 4)","(marine, 14)","(pelletan, 13)","(thomson, 12)","(tuberculose, 5)","(bord, 12)","(docteur, 11)"
1,"(hygiène, 6)","(marine, 5)","(kitasato, 4)","(gros, 7)","(hygiène, 4)","(bord, 10)","(eau, 9)","(gaston, 12)","(thomson, 4)","(marine, 8)","(bord, 7)"
2,"(islande, 5)","(médicale, 5)","(paludisme, 4)","(service, 6)","(médecine, 4)","(hôpitaux, 10)","(marine, 8)","(bord, 9)","(pulmonaire, 4)","(foie, 6)","(cas, 6)"
3,"(secours, 5)","(étiologie, 5)","(traitement, 4)","(statistique, 5)","(corps, 3)","(pelletan, 9)","(hôpital, 7)","(hygiène, 7)","(hôpital, 4)","(traduit, 4)","(abcès, 6)"
4,"(blessés, 5)","(béribéri, 4)","(médical, 4)","(fièvre, 5)","(campagne, 3)","(contre, 8)","(hôpitaux, 7)","(maritime, 6)","(cas, 4)","(nouveau, 4)","(étude, 6)"
5,"(cas, 5)","(statistique, 4)","(bacille, 4)","(maurel, 4)","(chine, 3)","(camille, 8)","(tuberculose, 6)","(décret, 5)","(machenaud, 4)","(abcès, 4)","(procédé, 5)"
6,"(pêcheurs, 5)","(hôpital, 4)","(fièvre, 4)","(bord, 4)","(service, 3)","(eau, 7)","(médecine, 6)","(cas, 5)","(temps, 4)","(navale, 4)","(croiseurs, 4)"
7,"(marine, 5)","(santé, 4)","(yersin, 3)","(blessés, 4)","(sac, 3)","(officiers, 7)","(bord, 6)","(service, 5)","(brunet, 4)","(conférence, 3)","(marine, 4)"
8,"(chastang, 4)","(indes, 3)","(cas, 3)","(santé, 4)","(santé, 3)","(service, 6)","(maritime, 6)","(croiseur, 5)","(juillet, 4)","(aspirants, 3)","(foie, 4)"
9,"(1897, 4)","(médecine, 3)","(émily, 2)","(gazeau, 3)","(cas, 3)","(lutte, 6)","(navale, 5)","(blessés, 4)","(marins, 4)","(gauche, 3)","(hôpital, 4)"


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np

# corpus = [txt_AMN,
#           txt_AHMC]
# vocabulary = ['Lèpre', 'Paludisme', 'Chine', 'Hygiène', 'Pasteur', 'Géographie',
#               'Béribéri', 'fièvre jaune']
# pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
#                  ('tfid', TfidfTransformer())]).fit(corpus)
# pipe['count'].transform(corpus).toarray()

# 3. Premières observations de NLP

In [ ]:
from python_to_doccano import pandas_to_doccano

In [ ]:
#AHMC : 
## en JSONL
output_pathAHMCjsonl = "./DOCCANO/doccano-input/AHMC_tokenized.jsonl"
pandas_to_doccano(df_spacyAHMC, nlp, output_pathAHMCjsonl)

#AMN
output_pathAMNjsonl = "./DOCCANO/doccano-input/AMN_tokenized.jsonl"
pandas_to_doccano(df_spacyAMN, nlp, output_pathAMNjsonl)

### initialisation

In [ ]:
with open(output_pathAHMCjsonl) as f:
    for line in f:
        print(f.readline())

In [ ]:
with open(output_pathAMNjsonl) as f:
    for line in f:
        print(f.readline())

In [ ]:
def show_ents(doc): 
    L=[]
    if doc.ents: 
        for ent in doc.ents: 
            annotations=ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))
            L.append(annotations)
            print(annotations)
            print("\n")

In [ ]:
with open (txtAMN_output) as txt:
    txtAMN = txt.read()

docAMN = nlp(txtAMN)
from spacy import displacy
displacy.render(docAMN, style = "ent",jupyter = True)

In [ ]:
with open (txtAHMC_output) as txt:
    txtAHMC = txt.read()
    
docAHMC=nlp(txtAHMC)
    
from spacy import displacy
displacy.render(docAHMC, style = "ent",jupyter = True)

Commentaire du NER avec l'algorithme par défault de Spacy :

* LOCALISATION :
    * Trop de `<GPE>` considérés comme `<LOC>` 
    * "Variété" pris comme `<LOC>`   

* Beaucoup de `<MISC>`

* Pb annotation comme "Paludisme" en `<ORG>` 

* La NER ne respecte pas la tokenisation => "Ssé" -"Mao" au lieu de "Ssé-Mao"

# 4. ENTRAINER LES DONNEES AVEC DOCCANO

1/Lancer l'Environnement virtuel (depuis ~)
* `cd Python-virtual_environment`
* `source ~/Python-virtual_environment/env-virtuel/bin/activate`
=> s'assurer qu'on a bien 'env-virtuel') au début de la ligne de commande

2/Lancer Doccano dans le terminal: 
* Ecrire "`Doccano`" dans le terminal
(si ça marche pas, faire un ` pip install doccano ` avant

aller sur : http://localhost:8000/. 
Vous pouvez vous loguer en tant qu'utilisateur "admin" en fournissant le mot de passe "password".

<b>NB : </b> 
Avant d'utiliser Doccano pour corriger les annotations automatiques faites par Spacy, il est nécéssaire de pouvoir les importer dans ce premier. Autrement dit, les annotations contenues dans l'objet Doc de Spacy seront transformées dans un format lisible par Doccano, il s'agit ici du format JSONL, une forme de JSON. Le résultat sera enregistré dans un fichier qui sera dans le même dossier.

## 4.1. Création du training set pour Doccano 

nous allons ici utiliser les fonctionnalités de scikitlearn pour diviser notre dataset en train, validate et test sets. 


Cross-validation: 
https://scikit-learn.org/stable/modules/cross_validation.html


Tuto train_set_split: https://www.youtube.com/watch?v=ZHPwPHkrT4I

une fois que j'ai délimité mon train set, je le redivise en x (=4 devrait être suffisant) => 4-fold Cross validation. 


In [ ]:
#preview : 
taille_AHMC=len(df_spacyAHMC)
taille_AMN=len(df_spacyAMN)

print("nombre d'articles des AHMC :",taille_AHMC)
print("nombre d'articles des AMN :",taille_AMN)
print("nombre d'articles au total:",taille_AHMC+taille_AMN)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
#random_state : permet d'avoir une 
#stratify : stratify for a specific column (?) 

train, test = train_test_split(dbf_spacyAHMC,test_size=0.33)

In [ ]:
len(test)

![Image](./images/training-validation-test-data-set.png)

In [ ]:
with open ("./DOCCANO/Doccano_AHMC230.txt",'r') as trainingset :
    doc_cano = nlp(trainingset.read())
show_ents(doc_cano)

In [ ]:
from spacy import displacy
displacy.render(doc_cano, style = "ent",jupyter = True)

In [ ]:
from spacy.tokenizer import Tokenizer
from spacy.lang.fr import French
tokenizer = Tokenizer(nlp.vocab)

transformation du fichier txt en JSONL avec les annotations NER du modèle par défault.

In [ ]:
from spacy.tokens import Span
import codecs
import json

with codecs.open("./DOCCANO/annotations_doc230.jsonl", 'w', encoding='utf8') as f:
    for sent in doc_cano.sents:
        text = sent.text
#         print(text)
        if(text.strip() != '' and not text.startswith("\n")): #pour enlever quelques lignes avec encodage problématique
            if re.search(r'[A-z]',text): #pour retirer les entrées contenant uniquement des ponctuations ou espaces.
                labels = []
                if (len(sent.ents) > 0):
                    for ent in sent.ents:
                        labels.append([ent.start_char-ent.sent.start_char, ent.end_char-ent.sent.start_char, ent.label_])
                #print(text)
                if (len(labels) > 0):
                    sentence = {"text": text, "labels": labels}    
                else:
                    sentence = {"text": text}
#                 print(json_string)
                json_string = json.dumps(sentence, ensure_ascii=False)
                f.write(json_string)
                f.write("\n")
f.close()
print("fichier créé : annotations_doc230.jsonl")

## 4.2. Correction des annotations sur Doccano 

### Données d'entrainement : 
Sélectionner 20% de mes données pour l'entrainement sur Doccano : 

J'ai au total 1137 articles pour les AHMC de 1898 à 1914. 

Labels à utiliser : 

> * `<LOC>` - Non-GPE locations, mountain ranges, bodies of water
> * `<GPE>` - Geopolitical entity 
> * `<PATH>` - Affection pathologique (paludisme, béribéri,..)
> * `<HYG>` - Relatifs 
> * `<ETHNO>` - Reference à une pratique ethnologique/ du terrain : Géographie médicale ;



### Retour d'expérience de Doccano : 

1) Pb de tokénisation - Beaucoup de mots mal découpés. Ex : 
* Clinique d'Outre \n -Mer.
* Tchong- \n King

2) Problème d'annotation quand on a des noms d'organisation `<ORG>` qui contiennent une   `<LOC>` ou `<PATH>`. Ex : 
* « Mission de la maladie du sommeil » 
* Hopital Lanessan
* Institut Pasteur de Brazzaville
===> J'ai identifié `<ORG>` et ça pourra par la suite être repris. 
===> Désambiguisation mais nécessité tout de même d'y avoir recours pour identifier de quoi et d'où on parle. 

3) Q° des médicaments/traitements/découvertes associées à un `<PER>` ou `<LOC> `.Ex: 
* « Sérum antipesteux de Yersin » →Pas annoté (car pas un type d'entité que j'ai cherché à identifier).

4) Les `<PATH>` avec nom de `<LOC>`. Ex : 
* « Bouton du Nil » 
===> Annoté comme `<PATH>` car il est connu que ces rattachements à des territoires ne reflète pas une réalité attachée à un territoire dédié.

5) Dérivés de nom de `<PATH>` pour désigner leur prophylaxie ou traitement. Ex : 
*  « antipaludique » 
===> Non annoté car n'est pas un non de `<PATH>` à proprement parler. 

6) `<PATH>` avec description 
· De même, comment gérer les noms de pahtologie à rallonge ? ex : « chylurie endémique pure »

7) Notions non annotées : pharmacolohie,microbiologie, parasitaires... Ex :
* hématozoaires
* anophèles

Pourrait pourtant être très important car permettrait de saisir une évolution épistémologique qui marque la première moitié du XXe siècle : découverte notamment du rôle de parasites comme les anophèles ou les puces pour certaines maladies.


<b color="red">IMPORTANT</b> : Je regrette de ne pas avoir fait de point sur les parasites. C’est un point tout de même essentiel du

NB : je suis à l’annotation 465 sur 625 et j’ai le sentiment que les données sont très répétitives. Les même termes reviennent extrêmmeent souvent.

Emplacement du document annoté:
    `"./DOCCANO/AHMC_to_1914-annotations-corrigees.jsonl"`

## 4.3. Import du JSONL Doccano pour Spacy et stockage dans TRAIN_DATA

In [ ]:
json_doc = "./DOCCANO/AHMC-230-annotations-corrigees.jsonl"
def train_data(json_doc):

    TRAIN_DATA = []

    with codecs.open(json_doc, "r", encoding="utf8") as f:
        lines = f.readlines()
    #     print(lines)
        for line in lines:
            line = json.loads(line)
    #         print(line)
            if "labels" in line:
                line["entities"] = line.pop("labels")
            else:
                line["entities"] = []
            ents = []
            for entity in line["entities"]:
                ents.append((entity[0], entity[1], entity[2]))
            TRAIN_DATA.append({"text": line["text"], "entities": ents})
            #json_string = json.dumps({"text": line["text"], "entities": ents}, ensure_ascii=False)
    f.close()
    return TRAIN_DATA

print(TRAIN_DATA)

## 4.3. Màj du modèle NER de départ 

### 4.3.1. ré-entraînement de fr_core_news_md avec les annotations stockées dans TRAIN_DATA

In [ ]:
from pathlib import Path
from spacy.util import minibatch, compounding 
from spacy.training import Example

ner = nlp.get_pipe("ner")

for annotations in TRAIN_DATA:
    print(annotations)
    for ent in annotations["entities"]:
        ner.add_label(ent[2])

### 4.3.2. Modifier la chaîne de départ par itération du nouveau modèle ner

In [ ]:
#On crée une varibale contenant l'ensemble des autres pipes: 
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(3):
        print("iteration: "+str(itn))
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            examples = []
            for ba in batch:
                examples.append(Example.from_dict(nlp.make_doc(ba["text"]), ba))
                nlp.update(examples)        
print("training is finished")    

In [ ]:
output_dir = Path("output_model")
if not output_dir.exists():
    output_dir.mkdir()
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

On teste le modèle enregistré. En apprentissage automatique, on ne teste pas un modèle sur le corpus sur lequel a été entrainé, ce pas permet de vérifier que l'entrainement s'est bien passé.

In [ ]:
print("Loading from", output_dir)
nlp_test = spacy.load(output_dir)
with open ("titres_articles_AHMC-to-1914.txt", "r", encoding="utf_8") as myfile_test:
    data_test = myfile_test.read()
doc_test = nlp_test(data_test)
for ent in doc_test.ents:
    print(ent.label_, ent.text)

In [ ]:
locationEntities = list(set([e.text for e in doc_test.ents if e.label_ == "LOC"]))
GPEEntities = list(set([e.text for e in doc_test.ents if e.label_ == "ORG"]))
PATHEntities = list(set([e.text for e in doc_test.ents if e.label_ == "PATH"]))
HYGEntities = list(set([e.text for e in doc_test.ents if e.label_ == "HYG"]))
SHSEntities = list(set([e.text for e in doc_test.ents if e.label_ == "SHS"]))

<b>Commentaire :</b>

Semble correct à priori. Je remarque tout de même que :
* Lévaditi, Yersin  notés en `<LOC>`
* Gaide, E. Froment, Ross en `<PATH>`
* annamite en `<PATH>`
* Cholécystite en `<SHS>`
* épizooties du bas en `<ORG>`
* 'Lombricose' identifié tantôt en `<LOC>`, tantôt en `<GPE>`
* Mong-Tseu en `<PATH>`
* Yang Tsé en `<PATH>`
 
Entités qui neconviennent qu'à moitié :
* entérite en `<PATH>` => C'est assez inexacte.
* Tonkin en `<LOC>` => Devrait être en `<GPE>`
* Inde anglaise au moyen du sérum antipesteux `<GPE>` => "Inde anglaise seulement)
* Épizootie de Surra en `<LOC>` => `<ZOO-ENTO>` + `<LOC>`


Application du modèle aux titres des AMN sur la même période :



==> ==> CONCLUSION :Mérite d'être entrainé à nouveau.

# 5. Application aux articles post 1919 pour estimer l'efficience du modèle 

## 5.1. Exploration résultats 

In [ ]:
df= pd.DataFrame()
with open ("../Memoire-M1/Structuration_donnees-revues/AHMC/AHMC_Medica/AHMC_articles-originaux_Termine2.tsv","r") as data:
    df = pd.read_csv(data,sep='\t')
#Ensemble des articles 
df_spacy2 = df.loc[df["revue_annee"]>= 1919] 
df_spacy2 = df.loc[df["revue_annee"]<=1930]
df_spacy2 = df_spacy2[["article_titre"]]
# print(df_spacy)

with open ("titres_articles_AHMC-1919-to-1930.txt","a") as articles_titre:
    for article in df_spacy2["article_titre"]:
        articles_titre.write(article)
        articles_titre.write("\n")

In [ ]:
print("Loading from", output_dir)
nlp_test = spacy.load(output_dir)
with open ("titres_articles_AHMC-1919-to-1930.txt", "r", encoding="utf_8") as myfile_test:
    data_test2 = myfile_test.read()
doc_test2 = nlp_test(data_test2)
for ent in doc_test2.ents:
    print(ent.label_, ent.text)

In [ ]:
displacy.render(doc_test2, style = "ent",jupyter = True)

In [ ]:
LOCEntities = list(set([e.text for e in doc_test2.ents if e.label_ == "LOC"]))
GPEEntities = list(set([e.text for e in doc_test2.ents if e.label_ == "ORG"]))
PATHEntities = list(set([e.text for e in doc_test2.ents if e.label_ == "PATH"]))
HYGEntities = list(set([e.text for e in doc_test2.ents if e.label_ == "HYG"]))
SHSEntities = list(set([e.text for e in doc_test2.ents if e.label_ == "SHS"]))

TailleEntities= {"LOC":len(LOCEntities),"GPE":len(GPEEntities),"PATH":len(PATHEntities),"HYG":len(HYGEntities),"SHS":len(SHSEntities)}

In [ ]:
TailleEntities